# Install Libraries

In [2]:
! pip install -q transformers accelerate bitsandbytes peft torch langchain sentence-transformers faiss-cpu langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 31.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━

# Import Libraries

In [5]:
import os
import pandas as pd
import re
from bs4 import BeautifulSoup
import time
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity

# HuggingFace & Quantization Imports
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# LangChain Imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory

# Global Configuration & Model Selection

In [19]:
# 1. LLM SELECTION
# Choose your LLM: 'llama31' or 'mistral'
LLM_NAME = 'mistral'

# 2. EMBEDDING SELECTION
# Choose your embedding model: 'e5-small' or 'minilm'
EMBEDDING_NAME = 'e5-small'

# --- LLM Paths and IDs ---
LLAMA_MODEL_PATH = "/kaggle/input/llama-3.1/transformers/8b-instruct/2"
MISTRAL_MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"

# --- Embedding IDs ---
E5_EMBEDDING_ID = "intfloat/e5-small-v2"
MINILM_EMBEDDING_ID = "sentence-transformers/all-MiniLM-L6-v2"

# Map selection to actual model IDs/Paths
if LLM_NAME == 'llama31':
    MODEL_ID = LLAMA_MODEL_PATH
elif LLM_NAME == 'mistral':
    MODEL_ID = MISTRAL_MODEL_ID
else:
    raise ValueError(f"Unknown LLM name: {LLM_NAME}")

if EMBEDDING_NAME == 'e5-small':
    EMBEDDING_ID = E5_EMBEDDING_ID
elif EMBEDDING_NAME == 'minilm':
    EMBEDDING_ID = MINILM_EMBEDDING_ID
else:
    raise ValueError(f"Unknown Embedding name: {EMBEDDING_NAME}")

print(f"Configuration set: LLM={LLM_NAME} ({MODEL_ID}), EMBEDDING={EMBEDDING_NAME} ({EMBEDDING_ID})")

Configuration set: LLM=mistral (mistralai/Mistral-7B-Instruct-v0.2), EMBEDDING=e5-small (intfloat/e5-small-v2)


# Global Variables

In [6]:
INPUT_BASE_PATH = "/kaggle/input/"
NEWS_DATASET_PATH = os.path.join(INPUT_BASE_PATH, "european-commission-news/eu_commission_news.csv")
LLAMA_MODEL_PATH = os.path.join(INPUT_BASE_PATH, "llama-3.1/transformers/8b-instruct/2")

In [7]:
OUTPUT_BASE_PATH = '/kaggle/working/'
CLEAN_DATASET_PATH = os.path.join(OUTPUT_BASE_PATH, "eu_commission_news_clean.csv")
CHUNKS_DATASET_PATH = os.path.join(OUTPUT_BASE_PATH, "eu_commission_news_chunks.csv")

# Load Data

In [20]:
df = pd.read_csv(NEWS_DATASET_PATH)

In [21]:
df.head()

,title,link,date,summary,description
0,Mounting risks threaten survival of wild Europ...,https://environment.ec.europa.eu/news/mounting...,11 October 2025,Nearly 100 additional wild bee species in Euro...,"Abu Dhabi, United Arab Emirates, 11 October 20..."
1,Energy Efficiency Directive: Advancing the EU’...,https://energy.ec.europa.eu/news/energy-effici...,10 October 2025,Tomorrow (11 October) marks the deadline for E...,Tomorrow (11 October) marks the deadline for E...
2,EU’s Leading Role Instrumental in advancing su...,https://research-and-innovation.ec.europa.eu/n...,10 October 2025,G20 reinforces international collaboration in ...,"The G20 Research, Science and Innovation Minis..."
3,President von der Leyen travels to the Western...,https://enlargement.ec.europa.eu/news/presiden...,10 October 2025,"The President of the European Commission, Ursu...","The President of the European Commission, Ursu..."
4,Fishing vessel engine power - new Commission g...,https://oceans-and-fisheries.ec.europa.eu/news...,10 October 2025,The European Commission has published two tech...,The European Commission has published two tech...


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        266 non-null    object
 1   link         266 non-null    object
 2   date         266 non-null    object
 3   summary      266 non-null    object
 4   description  266 non-null    object
dtypes: object(5)
memory usage: 10.5+ KB


# Data Cleaning

In [23]:
# Define a function to clean text
def clean_text(text):
    if pd.isna(text):
        return ""
    # Remove HTML tags
    text = BeautifulSoup(str(text), "html.parser").get_text()
    # Remove non-UTF characters
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    # Remove extra spaces and newlines
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply cleaning to relevant text columns
text_columns = ['title', 'summary', 'description']  # adjust based on your actual columns
for col in text_columns:
    if col in df.columns:
        df[col] = df[col].apply(clean_text)

# Save the cleaned data
df.to_csv(CLEAN_DATASET_PATH, index=False)

# Preview cleaned data
df.head()

,title,link,date,summary,description
0,Mounting risks threaten survival of wild Europ...,https://environment.ec.europa.eu/news/mounting...,11 October 2025,Nearly 100 additional wild bee species in Euro...,"Abu Dhabi, United Arab Emirates, 11 October 20..."
1,Energy Efficiency Directive: Advancing the EU ...,https://energy.ec.europa.eu/news/energy-effici...,10 October 2025,Tomorrow (11 October) marks the deadline for E...,Tomorrow (11 October) marks the deadline for E...
2,EU s Leading Role Instrumental in advancing su...,https://research-and-innovation.ec.europa.eu/n...,10 October 2025,G20 reinforces international collaboration in ...,"The G20 Research, Science and Innovation Minis..."
3,President von der Leyen travels to the Western...,https://enlargement.ec.europa.eu/news/presiden...,10 October 2025,"The President of the European Commission, Ursu...","The President of the European Commission, Ursu..."
4,Fishing vessel engine power - new Commission g...,https://oceans-and-fisheries.ec.europa.eu/news...,10 October 2025,The European Commission has published two tech...,The European Commission has published two tech...


# Text Chunking

In [24]:
# Combine relevant text fields into one
df["text"] = df[["title", "summary", "description"]].fillna("").agg(" ".join, axis=1)

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,      # ~300-400 tokens
    chunk_overlap=100,    # preserve context
    length_function=len
)

# Create chunks
chunks = []
for i, row in df.iterrows():
    splits = text_splitter.split_text(row["text"])
    for j, chunk in enumerate(splits):
        chunks.append({
            "id": f"{i}_{j}",
            "source_id": i,
            "title": row.get("title", ""),
            "date": row.get("date", ""),
            "url": row.get("url", ""),
            "text_chunk": chunk
        })

# Convert to DataFrame
chunks_df = pd.DataFrame(chunks)

# Save chunks for embedding
chunks_df.to_csv(CHUNKS_DATASET_PATH, index=False)

print(f"✅ Created {len(chunks_df)} text chunks.")
chunks_df.head()

✅ Created 684 text chunks.


,id,source_id,title,date,url,text_chunk
0,0_0,0,Mounting risks threaten survival of wild Europ...,11 October 2025,,Mounting risks threaten survival of wild Europ...
1,0_1,0,Mounting risks threaten survival of wild Europ...,11 October 2025,,"reveal serious challenges, with threats mounti..."
2,0_2,0,Mounting risks threaten survival of wild Europ...,11 October 2025,,are at risk of extinction. That compares to 77...
3,0_3,0,Mounting risks threaten survival of wild Europ...,11 October 2025,,pollinators are largely dependent on tradition...
4,0_4,0,Mounting risks threaten survival of wild Europ...,11 October 2025,,"such as carpenter bees, are benefiting from wa..."


# Embedding Init & FAISS Creation

In [25]:
# Initialize local embedding model based on global selection
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_ID)

# Create a FAISS vector store from text chunks
vectorstore = FAISS.from_texts(
    texts=chunks_df["text_chunk"].tolist(),
    embedding=embeddings,
    metadatas=[
        {"title": t, "date": d, "url": u}
        for t, d, u in zip(chunks_df["title"], chunks_df["date"], chunks_df["url"])
    ]
)

# Save FAISS index locally (include embedding name in filename)
vectorstore.save_local(f"faiss_index_{EMBEDDING_NAME}")

print(f"✅ Local embeddings ({EMBEDDING_NAME}) generated and FAISS index saved successfully.")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

✅ Local embeddings (e5-small) generated and FAISS index saved successfully.


# Load FAISS and Build Retriever

In [26]:
# Load FAISS vectorstore
vectorstore = FAISS.load_local(
    f"faiss_index_{EMBEDDING_NAME}", # Use dynamic filename
    embeddings,
    allow_dangerous_deserialization=True
)

# Build retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 5}) # top-5 relevant chunks

# LLM Initialization and QA Chain Setup

In [27]:
# 1. Define Quantization Config (Needed for both Llama and Mistral for T4 GPU)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# 2. Load Model and Tokenizer dynamically (uses the MODEL_ID defined in Cell 1)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto"
)

# Ensure padding token is set for generation consistency
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 3. Create HuggingFace pipeline
local_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    device_map="auto"
)

# 4. Wrap pipeline in LangChain LLM object
llm = HuggingFacePipeline(pipeline=local_pipe)

# 5. Setup RAG Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

print(f"✅ RAG Pipeline initialized with LLM: {LLM_NAME}.")

query = "What are the key initiatives of the European Commission in digital policy?"
result = qa_chain.run(query)

print("\n--- TEST QUERY RESULT ---")
print(result)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


✅ RAG Pipeline initialized with LLM: mistral.

--- TEST QUERY RESULT ---
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

EU at the forefront of AI-driven research and scientific innovation. At its centre is RAISE - the Resource for AI Science in Europe - a virtual European institute that pools and coordinates AI resources for developing AI and applying it in science. Strategic actions include Back in April 2025, the Commission launched the AI Continent Action Plan, a plan that set the path for Europe to become a global leader in AI. The Apply AI and the AI in Science strategies are the next step in delivering this ambition and in positioning the EU to accelerate the use of AI in key sectors and science. For more information Press release - Commission launches two strategies to speed up AI uptake in European industry and science Apply AI Strategy AI in Science Strategy

Co

# Conversational Retrieval Chain

In [28]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory
)

response = conversation_chain({"question": "What did the EU do about AI regulations?"})
print(response["answer"])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Keeping European industry and science at the forefront of AI Learn how two EU strategies are putting industry and science at the forefront of artificial intelligence helping Europe become the global leader in trustworthy AI Artificial Intelligence (AI) is transforming how businesses operate, reshaping public services, and revolutionising science. AI has the potential to improve our lives in many ways. As the global race to harness its potential heats up, the European Commission has put forward two strategies that will help Europe stay ahead in AI industry and science respectively. The Apply AI Strategy sets out how to speed up the use of AI in key industries and the public sector. It will be used to help the EU unlock its societal benefits - from more accurate healthcare diagnoses to enhancing the efficiency and accessibilit

# RETRIEVAL ANALYSIS (SIMILARITY CHECK)

In [29]:
query = "What did the European Commission announce about AI policy?"
query_embedding = embeddings.embed_query(query)

# Retrieve top 5 chunks
docs = retriever.get_relevant_documents(query)

# Compute cosine similarity for each retrieved doc
for i, doc in enumerate(docs):
    doc_emb = embeddings.embed_query(doc.page_content)
    sim = cosine_similarity([query_embedding], [doc_emb])[0][0]
    print(f"Doc {i+1} similarity: {sim:.3f}")
    print(f"→ {doc.metadata.get('title')}")
    print()

Doc 1 similarity: 0.880
→ Keeping European industry and science at the forefront of AI

Doc 2 similarity: 0.865
→ Keeping European industry and science at the forefront of AI

Doc 3 similarity: 0.863
→ Boosting the use of AI will make strategic industries more competitive

Doc 4 similarity: 0.855
→ AI is a strategic tool to improve scientific research

Doc 5 similarity: 0.849
→ AI is a strategic tool to improve scientific research



# Automatic Evaluation (Latency & Scoring)

In [30]:
# Example evaluation queries
evaluation_questions = [
    "What did the European Commission announce about climate policy?",
    "What are the recent initiatives on digital transformation?",
    "Did the European Commission discuss AI regulations?",
    "What were the main economic measures mentioned recently?",
    "What did the EU say about energy and sustainability?"
]

results = []

for query in evaluation_questions:
    print(f"\n🧩 Evaluating query: {query}")
    start = time.time()

    # Step 1. Retrieve documents
    retrieved_docs = retriever.get_relevant_documents(query)
    retrieved_texts = [doc.page_content for doc in retrieved_docs]
    combined_context = " ".join(retrieved_texts)

    # Step 2. Generate answer using the local RAG pipeline
    answer = qa_chain.run(query)

    end = time.time()

    # Step 3. Compute embeddings
    q_emb = embeddings.embed_query(query)
    a_emb = embeddings.embed_query(answer)
    c_emb = embeddings.embed_query(combined_context)

    # Step 4. Compute similarity scores
    sim_q_a = cosine_similarity([q_emb], [a_emb])[0][0]          # Relevance
    sim_c_a = cosine_similarity([c_emb], [a_emb])[0][0]          # Faithfulness
    conciseness = len(answer) / max(len(combined_context), 1)    # Conciseness proxy
    completeness = len(retrieved_texts) / 5                      # How many chunks retrieved (0–1)

    results.append({
        "query": query,
        "answer": answer,
        "relevance_score": round(sim_q_a, 3),
        "faithfulness_score": round(sim_c_a, 3),
        "conciseness_score": round(conciseness, 3),
        "completeness_score": round(completeness, 3),
        "response_time_sec": round(end - start, 2)
    })

# Convert to DataFrame
eval_auto_df = pd.DataFrame(results)

# Compute average scores
avg_scores = eval_auto_df[["relevance_score", "faithfulness_score", "conciseness_score", "completeness_score"]].mean()
print("\n✅ Average Automatic Evaluation Scores:")
print(avg_scores)

# Save results for report
eval_auto_df.to_csv("rag_auto_evaluation.csv", index=False)
print("\n📁 Saved: rag_auto_evaluation.csv")

# Display preview
eval_auto_df.head()


🧩 Evaluating query: What did the European Commission announce about climate policy?

🧩 Evaluating query: What are the recent initiatives on digital transformation?

🧩 Evaluating query: Did the European Commission discuss AI regulations?

🧩 Evaluating query: What were the main economic measures mentioned recently?

🧩 Evaluating query: What did the EU say about energy and sustainability?

✅ Average Automatic Evaluation Scores:
relevance_score       0.8406
faithfulness_score    0.9872
conciseness_score     1.1766
completeness_score    1.0000
dtype: float64

📁 Saved: rag_auto_evaluation.csv


,query,answer,relevance_score,faithfulness_score,conciseness_score,completeness_score,response_time_sec
0,What did the European Commission announce abou...,Use the following pieces of context to answer ...,0.844,0.990,1.171,1.0,23.18
1,What are the recent initiatives on digital tra...,Use the following pieces of context to answer ...,0.819,0.982,1.194,1.0,25.97
2,Did the European Commission discuss AI regulat...,Use the following pieces of context to answer ...,0.848,0.990,1.134,1.0,19.47
3,What were the main economic measures mentioned...,Use the following pieces of context to answer ...,0.831,0.987,1.208,1.0,26.79
4,What did the EU say about energy and sustainab...,Use the following pieces of context to answer ...,0.861,0.987,1.176,1.0,22.40
